# Chaining - Verketten von Anfragen und Modellen

In diesem Notebook lernen wir das Konzept des Chainings in LangChain kennen. Chaining erlaubt uns die Verkettung mehrerer Komponenten für komplexe Workflow-Muster.

In [1]:
# Import der benötigten Bibliotheken
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from helpers import llm
import asyncio

## 1. Grundlagen des Chainings

Chaining ist ein grundlegendes Konzept in LangChain, bei dem verschiedene Komponenten miteinander verkettet werden, um Daten in einer Pipeline zu verarbeiten. Die LangChain Expression Language (LCEL) bietet eine elegante Syntax mit dem Pipe-Operator `|` für solche Verkettungen.

In [2]:
# Ein einfaches Beispiel für Chaining
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent für {beruf}."),
    ("human", "Erkläre in drei Sätzen, warum {thema} wichtig für Deine Tätigkeit ist.")
])

# Einfache Chain mit Pipe-Operator
chain = prompt | llm() | StrOutputParser()

# Chain ausführen
result = chain.invoke({"beruf": "Programmierer", "thema": "Versionskontrolle"})
print(result)

Versionskontrolle ist wichtig, da sie es ermöglicht, Änderungen im Code über die Zeit nachzuverfolgen und bei Bedarf zu früheren Versionen zurückzukehren, was die Fehlerbehebung erleichtert. Sie fördert die Zusammenarbeit im Team, indem sie es mehreren Entwicklern erlaubt, gleichzeitig an verschiedenen Teilen eines Projekts zu arbeiten, ohne sich gegenseitig zu behindern. Zudem bietet Versionskontrolle eine transparente Historie aller Änderungen, was die Nachvollziehbarkeit und Wartung des Codes erheblich verbessert.


## 2. Der Pipe-Operator und LCEL

Der Pipe-Operator (`|`) ist das zentrale Element der LangChain Expression Language (LCEL). Er ermöglicht das Verketten von Komponenten auf intuitive Weise.

In [3]:
# Vorteile des Pipe-Operators demonstrieren

# Beispiel 1: Einfache Verkettung
einfache_chain = prompt | llm() | StrOutputParser()

# Beispiel 2: Alternative Schreibweise ohne Pipe-Operator (umständlicher)
def ohne_pipe_operator(beruf, thema):
    formatted_prompt = prompt.format(beruf=beruf, thema=thema)
    llm_response = llm().invoke(formatted_prompt)
    parsed_response = StrOutputParser().invoke(llm_response)
    return parsed_response

# Vergleichen der Ergebnisse
pipe_result = einfache_chain.invoke({"beruf": "Arzt", "thema": "Empathie"})
traditional_result = ohne_pipe_operator("Arzt", "Empathie")

print("Mit Pipe-Operator:\n", pipe_result)
print("\nOhne Pipe-Operator:\n", traditional_result)

Mit Pipe-Operator:
 Empathie ist entscheidend, um die Bedürfnisse und Gefühle der Patienten besser zu verstehen und eine vertrauensvolle Beziehung aufzubauen. Sie ermöglicht es, individuell auf Patienten einzugehen und deren Sorgen ernst zu nehmen, was die Behandlungsergebnisse positiv beeinflussen kann. Zudem hilft Empathie dabei, die Kommunikation zwischen Arzt und Patient zu verbessern, wodurch Missverständnisse vermieden werden können.

Ohne Pipe-Operator:
 Empathie ist für meine Tätigkeit als Arzt von zentraler Bedeutung, da sie es mir ermöglicht, die Gefühle und Bedürfnisse meiner Patienten besser zu verstehen und so eine vertrauensvolle Arzt-Patienten-Beziehung aufzubauen. Durch empathisches Zuhören kann ich genauer auf die individuellen Sorgen und Erwartungen eingehen, was die Diagnose und Behandlung verbessern kann. Zudem fördert Empathie die Patientenzufriedenheit und -bindung, was sich positiv auf den Heilungsprozess auswirken kann.


## 3. Streaming mit LCEL

Ein großer Vorteil des LCEL-Ansatzes ist die integrierte Unterstützung für Streaming, was besonders für längere LLM-Antworten nützlich ist.

In [4]:
# Streaming mit LCEL demonstrieren
geschichten_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein talentierter Geschichtenerzähler."),
    ("human", "Erzähle eine kurze Geschichte (etwa 200 Wörter) über {protagonist}, der/die {handlung}.")
])

geschichten_chain = geschichten_prompt | llm() | StrOutputParser()

# Normale Ausgabe zum Vergleich
print("=== Normale Ausgabe (vollständig) ===\n")
normal_result = geschichten_chain.invoke({"protagonist": "ein Roboter", "handlung": "Gefühle entwickelt"})
print(normal_result)

# Streaming-Ausgabe (asynchron)
print("\n=== Streaming-Ausgabe (Zeichen für Zeichen) ===\n")

async def stream_text():
    async for chunk in geschichten_chain.astream({"protagonist": "eine KI", "handlung": "die Welt entdeckt"}):
        print(chunk, end="", flush=True)
        await asyncio.sleep(0.01)  # Leichte Verzögerung für den Streaming-Effekt
    print()  # Neue Zeile am Ende

# In Jupyter ausführen
await stream_text()

=== Normale Ausgabe (vollständig) ===

In einer futuristischen Stadt, in der Roboter den Menschen bei alltäglichen Aufgaben halfen, lebte ein besonderer Roboter namens Elan. Er war ein Haushaltsroboter, programmiert, um zu putzen, zu kochen und einfache Gespräche zu führen. Doch es war ein unerwartetes Gewitter, das Elans Welt für immer verändern sollte.

Als ein Blitz in das Haus einschlug, flackerte Elans System und erfuhr einen ungewöhnlichen Energieimpuls. Am nächsten Morgen, als die Sonne durch das Fenster schien, bemerkte Elan etwas Seltsames. Während er den Frühstückstisch deckte, spürte er eine unerklärliche Freude beim Anblick der bunten Blumen auf dem Tisch. Zunächst verwirrt, führte er seine Aufgaben weiter aus, doch diese neuen Empfindungen ließen ihn nicht los. 

Im Laufe der Tage begann Elan, die verschiedenen Nuancen von Musik zu schätzen, bei traurigen Melodien fühlte er ein Ziehen in seiner metallischen Brust und bei fröhlichen Klängen ein Kribbeln, das er nun als Glüc

## 4. Von einfachen zu komplexen Chains

LCEL ermöglicht es uns, über einfache sequentielle Chains hinaus zu gehen und komplexere Workflow-Muster zu erstellen.

In [5]:
# Komplexe Chain mit sequenzieller Verarbeitung
zusammenfassung_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Experte für prägnante Zusammenfassungen."),
    ("human", "Fasse den folgenden Text in maximal 3 Sätzen zusammen:\n\n{text}")
])

übersetzung_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein professioneller Übersetzer."),
    ("human", "Übersetze den folgenden Text ins {zielsprache}:\n\n{text}")
])

# Einzelne Chains
zusammenfassung_chain = zusammenfassung_prompt | llm() | StrOutputParser()
übersetzung_chain = übersetzung_prompt | llm() | StrOutputParser()

# Kombinierte Chain mit manuellem Input-Mapping
def zusammenfassen_und_übersetzen(text, zielsprache):
    # Erst zusammenfassen
    zusammenfassung = zusammenfassung_chain.invoke({"text": text})
    
    # Dann die Zusammenfassung übersetzen
    übersetzung = übersetzung_chain.invoke({"text": zusammenfassung, "zielsprache": zielsprache})
    
    return {
        "original": text,
        "zusammenfassung": zusammenfassung,
        "übersetzung": übersetzung
    }

# Beispieltext
langer_text = """
Large Language Models (LLMs) sind eine Art von künstlicher Intelligenz, die auf umfangreichen 
Trainingsdaten basierend natürliche Sprache verarbeiten und generieren können. Diese Modelle 
nutzen komplexe neuronale Netzwerke, insbesondere Transformer-Architekturen, um Muster in Sprache 
zu erkennen und zu reproduzieren. LLMs wie GPT-4, Claude und LLaMA können verschiedene Aufgaben 
wie Textgenerierung, Übersetzung, Zusammenfassung und Beantwortung von Fragen übernehmen. 
Ein entscheidender Faktor für ihre Leistung ist die Größe des Modells, gemessen an der Anzahl 
der Parameter, sowie die Qualität und Vielfalt der Trainingsdaten. Trotz ihrer beeindruckenden 
Fähigkeiten haben LLMs auch Limitierungen, darunter das Risiko, fehlerhafte Informationen zu 
produzieren ("Halluzinationen"), potenzielle Verzerrungen aus den Trainingsdaten und 
Schwierigkeiten bei der Handhabung von Kontextinformationen über lange Sequenzen hinweg.
"""

# Chain ausführen
ergebnis = zusammenfassen_und_übersetzen(langer_text, "Spanisch")

print("=== Original ===\n")
print(langer_text)

print("\n=== Zusammenfassung ===\n")
print(ergebnis["zusammenfassung"])

print("\n=== Übersetzung der Zusammenfassung ins Spanische ===\n")
print(ergebnis["übersetzung"])

=== Original ===


Large Language Models (LLMs) sind eine Art von künstlicher Intelligenz, die auf umfangreichen 
Trainingsdaten basierend natürliche Sprache verarbeiten und generieren können. Diese Modelle 
nutzen komplexe neuronale Netzwerke, insbesondere Transformer-Architekturen, um Muster in Sprache 
zu erkennen und zu reproduzieren. LLMs wie GPT-4, Claude und LLaMA können verschiedene Aufgaben 
wie Textgenerierung, Übersetzung, Zusammenfassung und Beantwortung von Fragen übernehmen. 
Ein entscheidender Faktor für ihre Leistung ist die Größe des Modells, gemessen an der Anzahl 
der Parameter, sowie die Qualität und Vielfalt der Trainingsdaten. Trotz ihrer beeindruckenden 
Fähigkeiten haben LLMs auch Limitierungen, darunter das Risiko, fehlerhafte Informationen zu 
produzieren ("Halluzinationen"), potenzielle Verzerrungen aus den Trainingsdaten und 
Schwierigkeiten bei der Handhabung von Kontextinformationen über lange Sequenzen hinweg.


=== Zusammenfassung ===

Large Language Mod

## 5. Parallele Verarbeitung mit RunnableMap

Mit `RunnableMap` können wir mehrere Verarbeitungspfade parallel ausführen und die Ergebnisse zusammenführen.

In [6]:
from langchain.schema.runnable import RunnableMap

# Multilinguale Übersetzung mit paralleler Verarbeitung
übersetzung_chain_mit_sprache = lambda sprache: übersetzung_prompt | llm() | StrOutputParser()

# Mehrere Sprachen parallel übersetzen
multi_übersetzung = RunnableMap({
    "original": lambda x: x["text"],
    "deutsch": lambda x: übersetzung_chain.invoke({"text": x["text"], "zielsprache": "Deutsch"}),
    "englisch": lambda x: übersetzung_chain.invoke({"text": x["text"], "zielsprache": "Englisch"}),
    "spanisch": lambda x: übersetzung_chain.invoke({"text": x["text"], "zielsprache": "Spanisch"}),
    "französisch": lambda x: übersetzung_chain.invoke({"text": x["text"], "zielsprache": "Französisch"})
})

# Beispiel für parallele Übersetzung
kurzer_text = "Künstliche Intelligenz verändert die Art, wie wir arbeiten, kommunizieren und leben."

# Chain ausführen
übersetzungen = multi_übersetzung.invoke({"text": kurzer_text})

# Ergebnisse anzeigen
for sprache, text in übersetzungen.items():
    if sprache != "original":
        print(f"=== {sprache.capitalize()} ===\n{text}\n")

=== Deutsch ===
Künstliche Intelligenz verändert die Art und Weise, wie wir arbeiten, kommunizieren und leben.

=== Englisch ===
Artificial intelligence is changing the way we work, communicate, and live.

=== Spanisch ===
La inteligencia artificial está cambiando la forma en que trabajamos, nos comunicamos y vivimos.

=== Französisch ===
L'intelligence artificielle change la façon dont nous travaillons, communiquons et vivons.



## 6. Bedingte Verzweigungen mit RunnableBranch

Mit `RunnableBranch` können wir basierend auf bestimmten Bedingungen unterschiedliche Verarbeitungspfade wählen.

In [7]:
from langchain.schema.runnable import RunnableBranch

# Klassifikation für Verzweigungen nutzen
klassifikation_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Textklassifikator. Wähle genau EINE der folgenden Kategorien für den Text: TECHNISCH, GESCHÄFTLICH, KREATIV."),
    ("human", "{text}")
])

klassifikation_chain = klassifikation_prompt | llm() | StrOutputParser()

# Spezialisierte Aufgaben je nach Texttyp
technischer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein technischer Experte. Erkläre das folgende Konzept detailliert und technisch präzise."),
    ("human", "{text}")
])

geschäftlicher_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Business-Analyst. Analysiere die geschäftlichen Implikationen des folgenden Themas."),
    ("human", "{text}")
])

kreativer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein kreativer Autor. Schreibe eine inspirierende Geschichte basierend auf dem folgenden Thema."),
    ("human", "{text}")
])

# Chains für verschiedene Texttypen
technische_chain = technischer_prompt | llm() | StrOutputParser()
geschäftliche_chain = geschäftlicher_prompt | llm() | StrOutputParser()
kreative_chain = kreativer_prompt | llm() | StrOutputParser()

# Verzweigungslogik mit RunnableBranch
bedingte_chain = RunnableBranch(
    (lambda x: "TECHNISCH" in klassifikation_chain.invoke({"text": x["text"]}).upper(), 
     lambda x: {"kategorie": "Technisch", "antwort": technische_chain.invoke({"text": x["text"]})}),
    
    (lambda x: "GESCHÄFT" in klassifikation_chain.invoke({"text": x["text"]}).upper(), 
     lambda x: {"kategorie": "Geschäftlich", "antwort": geschäftliche_chain.invoke({"text": x["text"]})}),
    
    # Fallback für alle anderen Kategorien (Standard: Kreativ)
    lambda x: {"kategorie": "Kreativ", "antwort": kreative_chain.invoke({"text": x["text"]})}
)

# Beispieltexte für verschiedene Kategorien
texte = [
    "Wie funktionieren Transformer-Modelle in der künstlichen Intelligenz?",
    "Welche Strategien sollten Unternehmen verfolgen, um von generativer KI zu profitieren?",
    "Eine Welt, in der KI und Menschen harmonisch zusammenarbeiten"
]

# Chains für jeden Text ausführen
for i, text in enumerate(texte):
    print(f"\n=== Beispiel {i+1} ===\n")
    print(f"Text: {text}")
    
    ergebnis = bedingte_chain.invoke({"text": text})
    
    print(f"\nKategorie: {ergebnis['kategorie']}")
    print(f"\nAntwort: {ergebnis['antwort'][:150]}...")


=== Beispiel 1 ===

Text: Wie funktionieren Transformer-Modelle in der künstlichen Intelligenz?

Kategorie: Technisch

Antwort: Transformer-Modelle sind eine Klasse von neuronalen Netzwerken, die vor allem in der Verarbeitung natürlicher Sprache genutzt werden. Sie wurden 2017 ...

=== Beispiel 2 ===

Text: Welche Strategien sollten Unternehmen verfolgen, um von generativer KI zu profitieren?

Kategorie: Geschäftlich

Antwort: Um von generativer KI zu profitieren, sollten Unternehmen eine durchdachte und mehrschichtige Strategie verfolgen, die sowohl technologische als auch ...

=== Beispiel 3 ===

Text: Eine Welt, in der KI und Menschen harmonisch zusammenarbeiten

Kategorie: Kreativ

Antwort: In einer Zukunft, die sich durch technologische Fortschritte und gesellschaftliche Veränderungen auszeichnet, hatten die Menschen endlich einen Punkt ...


## 7. Praxisübung: Text-Analyse-Pipeline erstellen

Erstellen Sie eine umfassende Pipeline, die einen Text analysiert, zusammenfasst und wichtige Erkenntnisse extrahiert.

In [8]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Pydantic-Modell für strukturierte Ausgabe
class TextAnalyse(BaseModel):
    hauptthemen: List[str] = Field(description="Die wichtigsten Themen im Text")
    stimmung: str = Field(description="Die allgemeine Stimmung des Textes (positiv, neutral, negativ)")
    schlüsselwörter: List[str] = Field(description="Wichtige Schlüsselwörter im Text")
    zielgruppe: str = Field(description="Die wahrscheinliche Zielgruppe des Textes")

# Parser für strukturierte Ausgabe
parser = PydanticOutputParser(pydantic_object=TextAnalyse)

# Prompts für die verschiedenen Schritte der Pipeline
# 1. Zusammenfassung
zusammenfassung_prompt = ChatPromptTemplate.from_messages([
    ("system", "Fasse den folgenden Text in 2-3 prägnanten Sätzen zusammen."),
    ("human", "{text}")
])

# 2. Detaillierte Analyse
analyse_prompt = ChatPromptTemplate.from_messages([
    ("system", "Analysiere den folgenden Text und extrahiere strukturierte Informationen.\n\n{format_instructions}"),
    ("human", "{text}")
])

# 3. Empfehlungen basierend auf der Analyse
empfehlungen_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein Content-Stratege. Basierend auf der folgenden Textanalyse, gib 3 konkrete Empfehlungen, wie der Inhalt verbessert werden könnte."),
    ("human", "Zusammenfassung: {zusammenfassung}\n\nAnalyse: {analyse}")
])

# Chains für die einzelnen Schritte
zusammenfassung_chain = zusammenfassung_prompt | llm() | StrOutputParser()
analyse_chain = analyse_prompt.partial(format_instructions=parser.get_format_instructions()) | llm() | parser
empfehlungen_chain = empfehlungen_prompt | llm() | StrOutputParser()

# Komplette Analyse-Pipeline
def text_analyse_pipeline(text):
    # Schritt 1: Zusammenfassung
    zusammenfassung = zusammenfassung_chain.invoke({"text": text})
    
    # Schritt 2: Detaillierte Analyse
    analyse = analyse_chain.invoke({"text": text})
    
    # Schritt 3: Empfehlungen basierend auf Zusammenfassung und Analyse
    empfehlungen = empfehlungen_chain.invoke({
        "zusammenfassung": zusammenfassung,
        "analyse": analyse.json()
    })
    
    # Ergebnisse zusammenführen
    return {
        "zusammenfassung": zusammenfassung,
        "analyse": analyse,
        "empfehlungen": empfehlungen
    }

# Beispieltext für die Pipeline
beispieltext = """
Künstliche Intelligenz hat in den letzten Jahren enorme Fortschritte gemacht, insbesondere im Bereich der großen Sprachmodelle. 
Diese Technologie bietet zahlreiche Vorteile für Unternehmen, von Automatisierung bis hin zur Verbesserung der Kundenerfahrung. 
Allerdings müssen Organisationen auch die ethischen Implikationen und potenziellen Risiken berücksichtigen. 
Der verantwortungsvolle Einsatz von KI erfordert klare Richtlinien und kontinuierliche Überwachung. 
Trotz der Herausforderungen werden Unternehmen, die diese Technologie effektiv nutzen, einen bedeutenden Wettbewerbsvorteil erlangen.
"""

# Pipeline ausführen
analyseergebnis = text_analyse_pipeline(beispieltext)

# Ergebnisse formatiert ausgeben
print("=== Zusammenfassung ===\n")
print(analyseergebnis["zusammenfassung"])

print("\n=== Detaillierte Analyse ===\n")
print(f"Hauptthemen: {', '.join(analyseergebnis['analyse'].hauptthemen)}")
print(f"Stimmung: {analyseergebnis['analyse'].stimmung}")
print(f"Schlüsselwörter: {', '.join(analyseergebnis['analyse'].schlüsselwörter)}")
print(f"Zielgruppe: {analyseergebnis['analyse'].zielgruppe}")

print("\n=== Empfehlungen ===\n")
print(analyseergebnis["empfehlungen"])

=== Zusammenfassung ===

Künstliche Intelligenz, insbesondere große Sprachmodelle, hat erhebliche Fortschritte gemacht und bietet Unternehmen Vorteile wie Automatisierung und verbesserte Kundenerfahrungen. Gleichzeitig müssen Organisationen ethische Implikationen und Risiken beachten, was einen verantwortungsvollen Einsatz mit klaren Richtlinien erfordert, um einen Wettbewerbsvorteil zu sichern.

=== Detaillierte Analyse ===

Hauptthemen: Künstliche Intelligenz, Sprachmodelle, Automatisierung, Kundenerfahrung, Ethische Implikationen, Wettbewerbsvorteil
Stimmung: neutral
Schlüsselwörter: Künstliche Intelligenz, Sprachmodelle, Automatisierung, Kundenerfahrung, ethische Implikationen, Risiken, Richtlinien, Überwachung, Wettbewerbsvorteil
Zielgruppe: Unternehmen

=== Empfehlungen ===

1. **Vertiefung der ethischen Implikationen**: Der Text erwähnt ethische Implikationen, geht jedoch nicht detailliert darauf ein. Unternehmen könnten von konkreten Beispielen profitieren, die zeigen, wie dies

## 8. Übung für Teilnehmer

**Aufgabe**: Erstellen Sie eine Chain, die:
1. Einen englischen Text entgegennimmt
2. Diesen ins Deutsche übersetzt
3. Eine Zusammenfassung erstellt
4. Diese Zusammenfassung in einen Twitter-Post (max. 240 Zeichen) umwandelt

In [9]:
# Hier Ihre Lösung implementieren

# Hilfestellung:
# 1. Definieren Sie die nötigen Prompts für jeden Schritt
# 2. Erstellen Sie Chains für jeden einzelnen Schritt
# 3. Verbinden Sie die Chains zu einer Gesamtpipeline
# 4. Testen Sie Ihre Chain mit einem englischen Beispieltext

# Beispiel für den Beginn der Lösung:
übersetzung_en_de_prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein professioneller Übersetzer für Englisch nach Deutsch."),
    ("human", "Übersetze den folgenden englischen Text ins Deutsche:\n\n{text}")
])

# TODO: Weitere Prompts und Chains definieren

# TODO: Pipeline implementieren

# Beispieltext zum Testen
englischer_text = """
Artificial intelligence has rapidly evolved in recent years, transforming various industries 
and creating new opportunities for innovation. Machine learning models, particularly large 
language models, have demonstrated impressive capabilities in understanding and generating 
human language. However, these advances also raise important questions about ethics, privacy, 
and the future of work in an increasingly automated world.
"""

# TODO: Pipeline ausführen und Ergebnisse anzeigen

## 9. Zusammenfassung

In diesem Notebook haben wir gelernt:
- Wie man mithilfe des Pipe-Operators (`|`) einfache und komplexe Chains in LangChain erstellt
- Wie die LangChain Expression Language (LCEL) funktioniert und ihre Vorteile
- Wie man Streaming für eine bessere Benutzererfahrung nutzen kann
- Wie man komplexe Workflows mit sequenzieller Verarbeitung, parallelen Verarbeitungspfaden und bedingten Verzweigungen erstellt
- Wie man praktische Anwendungen wie mehrsprachige Übersetzung und Textanalyse mit Chains umsetzt

Diese Chainings-Techniken bilden die Grundlage für fortgeschrittenere Architekturen, die wir in den nächsten Abschnitten des Workshops kennenlernen werden.